In [1]:
#--------------------------------
## IMPORTS
#--------------------------------
import sys
import os

# set the path to find the modules
#sys.path.insert(0, '../src/') #use relative path
os.chdir("../005_src")

from config import *

----------------------------------------
Packages import successful
Variables import successful
Helper Functions import successful
Functions import successful
Functions xml import successful
Functions graph import successful
Functions GCN import successful
Functions Plotting import successful
Functions Data Adjustments import successful
----------------------------------------
root directory: /storage/remote/atcremers50/ss21_multiagentcontrol/
input directory: /storage/remote/atcremers50/ss21_multiagentcontrol/004_data/
output directory: /storage/remote/atcremers50/ss21_multiagentcontrol/006_model_output/
----------------------------------------



In [2]:
#--------------------------------
## WARNINGS MUTED
#--------------------------------
pd.options.mode.chained_assignment = None
plt.rcParams.update({'figure.max_open_warning': 0})


In [45]:
### old inputs with intention right, left, u turn
inputs = [#"20210710-20h38m27s_timesteps14930_ec3500_em7000",
          "20210711-17h59m44s_timesteps30000_ec3500_em7000",
          #"20210710-11h46m35s_timesteps200_ec3500_em7000",
          #"20210724-19h49m31s_timesteps150000_ec3500_em7000",
          #"20210725-16h24m21s_timesteps200_ec3500_em7000"
         ]

START = 0
END = 200

listdf_inputs = []
for input_file_name in inputs:
    print ()
    print (input_file_name)
    df_all,df_selected = select_rows_from_data(input_file_name,drop_col = True)
    df_input = adjust_columns(df_selected)
    df_input["num_veh_tot"] = df_input.apply (lambda row: count_num_veh(row), axis=1)
    df_input["num_veh_moving"] = df_input.apply (lambda row: count_num_veh_moving(row), axis=1)
    df_input["num_veh_still"] = df_input.apply (lambda row: count_num_veh_still(row), axis=1)

    #print (df_input[["data_pos_zc","data_y_zc",
                     #"data_y_delta","Still_vehicle",
                     #"num_veh_tot","num_veh_moving",
                     #"num_veh_still","all_veh_moving"]])

    #choose min max or standardization
    df_input_standard_normalized=standard_normalize_columns(df_input)
    #df_input_standard_normalized=min_max_normalize_columns(df_input)
    listdf_inputs.append(df_input)
    
big_fat_df = pd.concat(listdf_inputs)


20210711-17h59m44s_timesteps30000_ec3500_em7000
---------------------------
<class 'pandas.core.frame.DataFrame'>
---------------------------


In [46]:
print (len(big_fat_df))

24976


In [47]:
mask = big_fat_df.all_veh_moving == True
df_only_moving = big_fat_df[mask]
print (len(df_only_moving))

657


In [48]:
mask = big_fat_df.num_veh_still == big_fat_df.num_veh_tot
df_only_still = big_fat_df[mask]
print (len(df_only_still))

5873


In [49]:
mask = big_fat_df.all_veh_moving == False
df_others = big_fat_df[mask]

mask =  df_others.num_veh_still != 0
df_others = df_others[mask]

print (len(df_others))

24319


In [50]:
min_amount=min(len(df_only_moving),len(df_only_still), len(df_others))
min_amount

657

In [51]:
#df_others_sampled = df_others.sample(n = len(df_only_moving))
#df_only_still_sampled = df_only_still.sample(n = len(df_only_moving))
df_others_sampled = df_others.sample(n = min_amount)
df_only_still_sampled = df_only_still.sample(n = min_amount)
df_only_moving = df_only_moving.sample(n = min_amount)

In [52]:
df_only_moving

,data_x,data_pos,data_edges,data_edges_renamed,data_y,data_edges_attr,training_row,data_pos_zc,Still_vehicle,data_y_zc,...,data_x_rad,data_y_delta,all_veh_moving,num_veh_tot,num_veh_moving,num_veh_still,data_x_rad_norm,data_pos_zc_norm,data_y_zc_norm,data_y_yaw_norm
6790,"[[0.0, 8.68, 3.0], [0.0, 5.94, 1.0], [0.0, 3.1...","[[101.6, 123.8], [101.6, 105.61], [101.6, 91.0...","[[671, 671, 679, 671, 679, 676, 678, 671, 677,...","[[0, 0, 5, 0, 5, 2, 4, 0, 3, 1, 0, 3, 1, 0, 5,...","[[101.6, 146.24, 0.0], [101.6, 124.67, 0.0], [...","[5.1, 5.7, 8.7, 6.9, 7.3999999999999995, 9.8, ...",True,"[[1.5999999999999943, 23.799999999999997], [1....","[[False], [False], [False], [False], [False], ...","[[1.5999999999999943, 46.24000000000001], [1.5...",...,"[[0.0, 8.68, 3.0], [0.0, 5.94, 1.0], [0.0, 3.1...","[[0.0, 22.440000000000012], [0.0, 19.060000000...",True,10,10,0,"[[-0.7172051656172352, 2.3365665226594756, 1.2...","[[0.23918710876975188, 1.5603396858387237], [0...","[[0.23804270402949773, 2.4691759148750005], [0...","[[-0.7172000119796783], [-0.7172000119796783],..."
7961,"[[0.0, 12.71, 1.0], [0.0, 0.11, 3.0], [0.0, 1....","[[101.6, 138.21], [101.6, 91.72], [101.6, 83.9...","[[797, 793, 794, 801, 804, 792, 792, 797, 805,...","[[3, 1, 2, 4, 6, 0, 0, 3, 7, 5, 3, 5, 1, 4, 6,...","[[101.6, 170.89, 0.0], [101.6, 91.78, 0.0], [1...","[5.6, 3.0, 4.6, 3.6, 8.3, 3.3, 3.4000000000000...",True,"[[1.5999999999999943, 38.21000000000001], [1.5...","[[False], [False], [False], [False], [False], ...","[[1.5999999999999943, 70.88999999999999], [1.5...",...,"[[0.0, 12.71, 1.0], [0.0, 0.11, 3.0], [0.0, 1....","[[0.0, 32.67999999999998], [0.0, 0.06000000000...",True,10,10,0,"[[-0.7172051656172352, 3.6023691988184305, -0....","[[0.23918710876975188, 2.1357149256959134], [0...","[[0.23804270402949773, 3.464907564698327], [0....","[[-0.7172000119796783], [-0.7172000119796783],..."
3630,"[[0.0, 8.31, 1.0], [0.0, 1.82, 3.0], [0.0, 3.2...","[[101.6, 112.33], [101.6, 90.67], [101.6, 80.7...","[[377, 362, 360, 378, 362, 364, 372, 360, 371,...","[[7, 1, 0, 8, 1, 2, 5, 0, 4, 3, 3, 1, 2, 0, 2,...","[[101.6, 135.1, 0.0], [101.6, 91.68, 0.0], [10...","[1.7, 4.3, 3.6, 2.7, 13.5, 7.8999999999999995,...",True,"[[1.5999999999999943, 12.329999999999998], [1....","[[False], [False], [False], [False], [False], ...","[[1.5999999999999943, 35.099999999999994], [1....",...,"[[0.0, 8.31, 1.0], [0.0, 1.82, 3.0], [0.0, 3.2...","[[0.0, 22.769999999999996], [0.0, 1.0100000000...",True,10,10,0,"[[-0.7172051656172352, 2.220351388620068, -0.4...","[[0.23918710876975188, 1.1023553693111765], [0...","[[0.23804270402949773, 2.0191779197824293], [0...","[[-0.7172000119796783], [-0.7172000119796783],..."
22354,"[[0.0, 15.94, 1.0], [0.0, 7.11, 3.0], [270.0, ...","[[101.6, 159.0], [101.6, 116.4], [108.36, 101....","[[2099, 2090, 2092, 2102, 2099, 2092, 2090, 20...","[[5, 0, 1, 8, 5, 1, 0, 2, 0, 2, 2, 4, 4, 0, 5,...","[[101.6, 196.57, 0.0], [101.6, 135.7, 0.0], [1...","[4.0, 2.5, 5.6, 3.7, 5.6, 5.1, 2.4, 4.5, 2.5, ...",True,"[[1.5999999999999943, 59.0], [1.59999999999999...","[[False], [False], [False], [False], [False], ...","[[1.5999999999999943, 96.57], [1.5999999999999...",...,"[[0.0, 15.94, 1.0], [0.0, 7.11, 3.0], [4.71238...","[[0.0, 37.56999999999999], [0.0, 19.2999999999...",True,10,10,0,"[[-0.7172051656172352, 4.616895909486773, -0.4...","[[0.23918710876975188, 2.9658364549555216], [0...","[[0.23804270402949773, 4.502245851410824], [0....","[[-0.7172000119796783], [-0.7172000119796783],..."
28082,"[[160.84, 0.01, 0.0], [167.1, 4.79, 3.0], [180...","[[99.9, 105.65], [98.4, 91.72], [98.4, 113.03]...","[[2675, 2669, 2673, 2673, 2671, 2663, 2670, 26...","[[6, 1, 4, 4, 3, 0, 2, 2, 1, 1, 0, 2, 5, 0, 3,...","[[99.91, 105.65, 160.72], [98.4, 77.89, 180.0]...","[4.8, 6.8, 9.1, 6.4, 9.0, 5.3, 4.8, 17.1, 5.0,...",True,"[[-0.09999999999999432, 5.650000000000006], [-...","[[False], [False], [False], [False], [False], ...","[[-0.09000000000000341, 5.650000000000006], [-...",...,"[[2.8071875689076795

In [53]:
df_only_still_sampled

,data_x,data_pos,data_edges,data_edges_renamed,data_y,data_edges_attr,training_row,data_pos_zc,Still_vehicle,data_y_zc,...,data_x_rad,data_y_delta,all_veh_moving,num_veh_tot,num_veh_moving,num_veh_still,data_x_rad_norm,data_pos_zc_norm,data_y_zc_norm,data_y_yaw_norm
3273,"[[0.0, 0.0, 1.0], [0.0, 0.0, 1.0], [0.0, 0.0, ...","[[101.6, 91.8], [101.6, 84.3], [101.6, 76.8], ...","[[321, 320, 327, 321, 324, 320, 320, 320, 331,...","[[1, 0, 4, 1, 2, 0, 0, 0, 7, 3, 2, 0, 2, 5, 0,...","[[101.6, 91.8, 0.0], [101.6, 84.3, 0.0], [101....","[4.800000000000001, 15.0, 3.3000000000000003, ...",True,"[[1.5999999999999943, -8.200000000000003], [1....","[[True], [True], [True], [True], [True], [True...","[[1.5999999999999943, -8.200000000000003], [1....",...,"[[0.0, 0.0, 1.0], [0.0, 0.0, 1.0], [0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",False,10,0,10,"[[-0.7172051656172352, -0.389777702913657, -0....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.23804270402949773, 0.2700833787134613], [0...","[[-0.7172000119796783], [-0.7172000119796783],..."
13684,"[[0.0, 0.0, 1.0], [0.0, 0.0, 1.0], [0.0, 0.0, ...","[[101.6, 91.8], [101.6, 84.3], [101.6, 76.8], ...","[[1311, 1308, 1311, 1317, 1310, 1316, 1305, 13...","[[4, 1, 4, 7, 3, 6, 0, 5, 7, 2, 0, 1, 0, 3, 8,...","[[101.6, 91.8, 0.0], [101.6, 84.3, 0.0], [101....","[3.8, 9.0, 3.8, 3.1, 4.1, 3.2, 7.6, 16.2, 8.6,...",True,"[[1.5999999999999943, -8.200000000000003], [1....","[[True], [True], [True], [True], [True], [True...","[[1.5999999999999943, -8.200000000000003], [1....",...,"[[0.0, 0.0, 1.0], [0.0, 0.0, 1.0], [0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",False,10,0,10,"[[-0.7172051656172352, -0.389777702913657, -0....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.23804270402949773, 0.2700833787134613], [0...","[[-0.7172000119796783], [-0.7172000119796783],..."
19374,"[[0.0, 0.0, 1.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...","[[101.6, 91.8], [101.6, 84.3], [101.6, 76.8], ...","[[1835, 1838, 1828, 1828, 1829, 1835, 1830, 18...","[[5, 7, 0, 0, 1, 5, 2, 4, 2, 3, 1, 5, 6, 2, 2,...","[[101.6, 91.8, 0.0], [101.6, 84.3, 0.0], [101....","[8.6, 2.8, 4.3, 5.5, 4.800000000000001, 3.3000...",True,"[[1.5999999999999943, -8.200000000000003], [1....","[[True], [True], [True], [True], [True], [True...","[[1.5999999999999943, -8.200000000000003], [1....",...,"[[0.0, 0.0, 1.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",False,10,0,10,"[[-0.7172051656172352, -0.389777702913657, -0....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.23804270402949773, 0.2700833787134613], [0...","[[-0.7172000119796783], [-0.7172000119796783],..."
6037,"[[0.0, 0.0, 1.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...","[[101.6, 91.8], [101.6, 84.3], [101.6, 76.8], ...","[[577, 569, 581, 579, 570, 570, 569, 569, 572,...","[[3, 0, 5, 4, 1, 1, 0, 0, 2, 4, 2, 6, 1, 3, 0,...","[[101.6, 91.8, 0.0], [101.6, 84.3, 0.0], [101....","[9.2, 5.5, 3.7, 4.9, 11.0, 4.800000000000001, ...",True,"[[1.5999999999999943, -8.200000000000003], [1....","[[True], [True], [True], [True], [True], [True...","[[1.5999999999999943, -8.200000000000003], [1....",...,"[[0.0, 0.0, 1.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",False,10,0,10,"[[-0.7172051656172352, -0.389777702913657, -0....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.23804270402949773, 0.2700833787134613], [0...","[[-0.7172000119796783], [-0.7172000119796783],..."
27958,"[[0.0, 0.0, 1.0], [0.0, 0.0, 1.0], [180.0, 0.0...","[[101.6, 91.8], [101.6, 84.3], [98.4, 108.2], ...","[[2653, 2644, 2644, 2652, 2648, 2644, 2648, 26...","[[3, 0, 0, 2, 1, 0, 1, 5, 0, 5, 0, 0, 4, 5, 3,...","[[101.6, 91.8, 0.0], [101.6, 84.3, 0.0], [98.4...","[5.9, 7.6, 13.0, 3.7, 11.0, 16.2, 6.6, 3.8, 3....",True,"[[1.5999999999999943, -8.200000000000003], [1....","[[True], [True], [True], [True], [True], [True...","[[1.5999999999999943, -8.200000000000003], [1....",...,"[[0.0, 0.0, 1.0

In [54]:
df_others_sampled 

,data_x,data_pos,data_edges,data_edges_renamed,data_y,data_edges_attr,training_row,data_pos_zc,Still_vehicle,data_y_zc,...,data_x_rad,data_y_delta,all_veh_moving,num_veh_tot,num_veh_moving,num_veh_still,data_x_rad_norm,data_pos_zc_norm,data_y_zc_norm,data_y_yaw_norm
22654,"[[0.0, 0.0, 1.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...","[[101.6, 91.8], [101.6, 84.3], [101.6, 76.8], ...","[[2124, 2131, 2121, 2124, 2130, 2123, 2123, 21...","[[2, 6, 0, 2, 5, 1, 1, 3, 3, 6, 2, 1, 1, 4, 4,...","[[101.6, 91.8, 0.0], [101.6, 84.3, 0.0], [101....","[5.9, 2.9, 4.3, 10.0, 3.3000000000000003, 8.4,...",True,"[[1.5999999999999943, -8.200000000000003], [1....","[[True], [True], [True], [True], [True], [True...","[[1.5999999999999943, -8.200000000000003], [1....",...,"[[0.0, 0.0, 1.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",False,10,0,10,"[[-0.7172051656172352, -0.389777702913657, -0....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.23804270402949773, 0.2700833787134613], [0...","[[-0.7172000119796783], [-0.7172000119796783],..."
3100,"[[0.0, 0.0, 1.0], [90.0, 0.0, 0.0], [90.0, 0.0...","[[101.6, 91.8], [91.8, 98.4], [84.3, 98.4], [7...","[[302, 320, 320, 319, 314, 318, 315, 317, 316,...","[[0, 7, 7, 6, 1, 5, 2, 4, 3, 1, 1, 5, 3, 2, 4,...","[[101.6, 91.8, 0.0], [91.8, 98.4, 90.0], [84.3...","[16.2, 3.2, 10.0, 6.6, 9.2, 3.8, 3.90000000000...",True,"[[1.5999999999999943, -8.200000000000003], [-8...","[[True], [True], [True], [True], [True], [True...","[[1.5999999999999943, -8.200000000000003], [-8...",...,"[[0.0, 0.0, 1.0], [1.5707963267948966, 0.0, 0....","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",False,10,0,10,"[[-0.7172051656172352, -0.389777702913657, -0....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.23804270402949773, 0.2700833787134613], [-...","[[-0.7172000119796783], [0.2740812514905948], ..."
6272,"[[90.0, 0.0, 0.0], [0.0, 0.0, 1.0], [90.0, 0.0...","[[91.8, 98.4], [101.6, 91.8], [84.3, 98.4], [9...","[[606, 603, 604, 611, 609, 604, 611, 610, 606,...","[[2, 0, 1, 5, 3, 1, 5, 4, 2, 0, 3, 8, 4, 2, 4,...","[[91.8, 98.4, 90.0], [101.6, 91.8, 0.0], [84.3...","[11.0, 7.6, 5.9, 5.9, 7.6, 6.199999999999999, ...",True,"[[-8.200000000000003, -1.5999999999999943], [1...","[[True], [True], [True], [True], [True], [True...","[[-8.200000000000003, -1.5999999999999943], [1...",...,"[[1.5707963267948966, 0.0, 0.0], [0.0, 0.0, 1....","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",False,10,0,10,"[[0.2781069606211392, -0.389777702913657, -1.3...","[[-0.3372375545373098, 0.5461459944873984], [0...","[[-0.35010771943437335, 0.5366890131489164], [...","[[0.2740812514905948], [-0.7172000119796783], ..."
28859,"[[0.0, 0.0, 1.0], [270.0, 0.0, 3.0], [270.0, 0...","[[101.6, 91.8], [108.2, 101.6], [115.7, 101.6]...","[[2756, 2754, 2750, 2752, 2752, 2757, 2750, 27...","[[3, 2, 0, 1, 1, 4, 0, 0, 1, 1, 0, 7, 1, 4, 4,...","[[101.6, 91.8, 0.0], [108.2, 101.6, 270.0], [1...","[8.5, 7.9, 5.9, 6.4, 6.199999999999999, 16.2, ...",True,"[[1.5999999999999943, -8.200000000000003], [8....","[[True], [True], [True], [True], [True], [Fals...","[[1.5999999999999943, -8.200000000000003], [8....",...,"[[0.0, 0.0, 1.0], [4.71238898038469, 0.0, 3.0]...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",False,10,2,8,"[[-0.7172051656172352, -0.389777702913657, -0....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.23804270402949773, 0.2700833787134613], [0...","[[-0.7172000119796783], [2.256643778431141], [..."
14067,"[[0.0, 0.0, 3.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...","[[101.6, 91.8], [101.6, 84.3], [101.6, 76.8], ...","[[1333, 1326, 1326, 1340, 1326, 1330, 1330, 13...","[[3, 0, 0, 5, 0, 1, 1, 6, 2, 1, 3, 3, 0, 1, 6,...","[[101.6, 91.8, 0.0], [101.6, 84.3, 0.0], [101....","[9.3, 16.2, 5.5, 3.3000000000000003, 13.0, 2.7...",True,"[[1.5999999999999943, -8.200000000000003], [1....","[[True], [True], [True], [True], [True], [True...","[[1.5999999999999943, -8.200000000000003], [1....",...,"[[0.0, 0.0, 3.0]

In [55]:
df_balanced = pd.concat([df_others_sampled, df_only_moving, df_only_still_sampled], sort=True)
df_balanced

,Still_vehicle,all_veh_moving,data_edges,data_edges_attr,data_edges_renamed,data_pos,data_pos_zc,data_pos_zc_norm,data_x,data_x_rad,...,data_y,data_y_delta,data_y_yaw,data_y_yaw_norm,data_y_zc,data_y_zc_norm,num_veh_moving,num_veh_still,num_veh_tot,training_row
22654,"[[True], [True], [True], [True], [True], [True...",False,"[[2124, 2131, 2121, 2124, 2130, 2123, 2123, 21...","[5.9, 2.9, 4.3, 10.0, 3.3000000000000003, 8.4,...","[[2, 6, 0, 2, 5, 1, 1, 3, 3, 6, 2, 1, 1, 4, 4,...","[[101.6, 91.8], [101.6, 84.3], [101.6, 76.8], ...","[[1.5999999999999943, -8.200000000000003], [1....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.0, 0.0, 1.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...","[[0.0, 0.0, 1.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...",...,"[[101.6, 91.8, 0.0], [101.6, 84.3, 0.0], [101....","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[-0.7172000119796783], [-0.7172000119796783],...","[[1.5999999999999943, -8.200000000000003], [1....","[[0.23804270402949773, 0.2700833787134613], [0...",0,10,10,True
3100,"[[True], [True], [True], [True], [True], [True...",False,"[[302, 320, 320, 319, 314, 318, 315, 317, 316,...","[16.2, 3.2, 10.0, 6.6, 9.2, 3.8, 3.90000000000...","[[0, 7, 7, 6, 1, 5, 2, 4, 3, 1, 1, 5, 3, 2, 4,...","[[101.6, 91.8], [91.8, 98.4], [84.3, 98.4], [7...","[[1.5999999999999943, -8.200000000000003], [-8...","[[0.23918710876975188, 0.28261535027799856], [...","[[0.0, 0.0, 1.0], [90.0, 0.0, 0.0], [90.0, 0.0...","[[0.0, 0.0, 1.0], [1.5707963267948966, 0.0, 0....",...,"[[101.6, 91.8, 0.0], [91.8, 98.4, 90.0], [84.3...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [1.5707963267948966], [1.5707963267948...","[[-0.7172000119796783], [0.2740812514905948], ...","[[1.5999999999999943, -8.200000000000003], [-8...","[[0.23804270402949773, 0.2700833787134613], [-...",0,10,10,True
6272,"[[True], [True], [True], [True], [True], [True...",False,"[[606, 603, 604, 611, 609, 604, 611, 610, 606,...","[11.0, 7.6, 5.9, 5.9, 7.6, 6.199999999999999, ...","[[2, 0, 1, 5, 3, 1, 5, 4, 2, 0, 3, 8, 4, 2, 4,...","[[91.8, 98.4], [101.6, 91.8], [84.3, 98.4], [9...","[[-8.200000000000003, -1.5999999999999943], [1...","[[-0.3372375545373098, 0.5461459944873984], [0...","[[90.0, 0.0, 0.0], [0.0, 0.0, 1.0], [90.0, 0.0...","[[1.5707963267948966, 0.0, 0.0], [0.0, 0.0, 1....",...,"[[91.8, 98.4, 90.0], [101.6, 91.8, 0.0], [84.3...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[1.5707963267948966], [0.0], [1.5707963267948...","[[0.2740812514905948], [-0.7172000119796783], ...","[[-8.200000000000003, -1.5999999999999943], [1...","[[-0.35010771943437335, 0.5366890131489164], [...",0,10,10,True
28859,"[[True], [True], [True], [True], [True], [Fals...",False,"[[2756, 2754, 2750, 2752, 2752, 2757, 2750, 27...","[8.5, 7.9, 5.9, 6.4, 6.199999999999999, 16.2, ...","[[3, 2, 0, 1, 1, 4, 0, 0, 1, 1, 0, 7, 1, 4, 4,...","[[101.6, 91.8], [108.2, 101.6], [115.7, 101.6]...","[[1.5999999999999943, -8.200000000000003], [8....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.0, 0.0, 1.0], [270.0, 0.0, 3.0], [270.0, 0...","[[0.0, 0.0, 1.0], [4.71238898038469, 0.0, 3.0]...",...,"[[101.6, 91.8, 0.0], [108.2, 101.6, 270.0], [1...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [4.71238898038469], [4.71238898038469]...","[[-0.7172000119796783], [2.256643778431141], [...","[[1.5999999999999943, -8.200000000000003], [8....","[[0.23804270402949773, 0.2700833787134613], [0...",2,8,10,True
14067,"[[True], [True], [True], [True], [True], [True...",False,"[[1333, 1326, 1326, 1340, 1326, 1330, 1330, 13...","[9.3, 16.2, 5.5, 3.3000000000000003, 13.0, 2.7...","[[3, 0, 0, 5, 0, 1, 1, 6, 2, 1, 3, 3, 0, 1, 6,...","[[101.6, 91.8], [101.6, 84.3], [101.6, 76.8], ...","[[1.5999999999999943, -8.200000000000003], [1....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.0, 0.0, 3.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...","[[0.0, 0.0, 3.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...",...,"[[101.6, 91.8, 0.0],

In [56]:
df_balanced = df_balanced.reset_index(drop= True)
df_balanced

,Still_vehicle,all_veh_moving,data_edges,data_edges_attr,data_edges_renamed,data_pos,data_pos_zc,data_pos_zc_norm,data_x,data_x_rad,...,data_y,data_y_delta,data_y_yaw,data_y_yaw_norm,data_y_zc,data_y_zc_norm,num_veh_moving,num_veh_still,num_veh_tot,training_row
0,"[[True], [True], [True], [True], [True], [True...",False,"[[2124, 2131, 2121, 2124, 2130, 2123, 2123, 21...","[5.9, 2.9, 4.3, 10.0, 3.3000000000000003, 8.4,...","[[2, 6, 0, 2, 5, 1, 1, 3, 3, 6, 2, 1, 1, 4, 4,...","[[101.6, 91.8], [101.6, 84.3], [101.6, 76.8], ...","[[1.5999999999999943, -8.200000000000003], [1....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.0, 0.0, 1.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...","[[0.0, 0.0, 1.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...",...,"[[101.6, 91.8, 0.0], [101.6, 84.3, 0.0], [101....","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[-0.7172000119796783], [-0.7172000119796783],...","[[1.5999999999999943, -8.200000000000003], [1....","[[0.23804270402949773, 0.2700833787134613], [0...",0,10,10,True
1,"[[True], [True], [True], [True], [True], [True...",False,"[[302, 320, 320, 319, 314, 318, 315, 317, 316,...","[16.2, 3.2, 10.0, 6.6, 9.2, 3.8, 3.90000000000...","[[0, 7, 7, 6, 1, 5, 2, 4, 3, 1, 1, 5, 3, 2, 4,...","[[101.6, 91.8], [91.8, 98.4], [84.3, 98.4], [7...","[[1.5999999999999943, -8.200000000000003], [-8...","[[0.23918710876975188, 0.28261535027799856], [...","[[0.0, 0.0, 1.0], [90.0, 0.0, 0.0], [90.0, 0.0...","[[0.0, 0.0, 1.0], [1.5707963267948966, 0.0, 0....",...,"[[101.6, 91.8, 0.0], [91.8, 98.4, 90.0], [84.3...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [1.5707963267948966], [1.5707963267948...","[[-0.7172000119796783], [0.2740812514905948], ...","[[1.5999999999999943, -8.200000000000003], [-8...","[[0.23804270402949773, 0.2700833787134613], [-...",0,10,10,True
2,"[[True], [True], [True], [True], [True], [True...",False,"[[606, 603, 604, 611, 609, 604, 611, 610, 606,...","[11.0, 7.6, 5.9, 5.9, 7.6, 6.199999999999999, ...","[[2, 0, 1, 5, 3, 1, 5, 4, 2, 0, 3, 8, 4, 2, 4,...","[[91.8, 98.4], [101.6, 91.8], [84.3, 98.4], [9...","[[-8.200000000000003, -1.5999999999999943], [1...","[[-0.3372375545373098, 0.5461459944873984], [0...","[[90.0, 0.0, 0.0], [0.0, 0.0, 1.0], [90.0, 0.0...","[[1.5707963267948966, 0.0, 0.0], [0.0, 0.0, 1....",...,"[[91.8, 98.4, 90.0], [101.6, 91.8, 0.0], [84.3...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[1.5707963267948966], [0.0], [1.5707963267948...","[[0.2740812514905948], [-0.7172000119796783], ...","[[-8.200000000000003, -1.5999999999999943], [1...","[[-0.35010771943437335, 0.5366890131489164], [...",0,10,10,True
3,"[[True], [True], [True], [True], [True], [Fals...",False,"[[2756, 2754, 2750, 2752, 2752, 2757, 2750, 27...","[8.5, 7.9, 5.9, 6.4, 6.199999999999999, 16.2, ...","[[3, 2, 0, 1, 1, 4, 0, 0, 1, 1, 0, 7, 1, 4, 4,...","[[101.6, 91.8], [108.2, 101.6], [115.7, 101.6]...","[[1.5999999999999943, -8.200000000000003], [8....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.0, 0.0, 1.0], [270.0, 0.0, 3.0], [270.0, 0...","[[0.0, 0.0, 1.0], [4.71238898038469, 0.0, 3.0]...",...,"[[101.6, 91.8, 0.0], [108.2, 101.6, 270.0], [1...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [4.71238898038469], [4.71238898038469]...","[[-0.7172000119796783], [2.256643778431141], [...","[[1.5999999999999943, -8.200000000000003], [8....","[[0.23804270402949773, 0.2700833787134613], [0...",2,8,10,True
4,"[[True], [True], [True], [True], [True], [True...",False,"[[1333, 1326, 1326, 1340, 1326, 1330, 1330, 13...","[9.3, 16.2, 5.5, 3.3000000000000003, 13.0, 2.7...","[[3, 0, 0, 5, 0, 1, 1, 6, 2, 1, 3, 3, 0, 1, 6,...","[[101.6, 91.8], [101.6, 84.3], [101.6, 76.8], ...","[[1.5999999999999943, -8.200000000000003], [1....","[[0.23918710876975188, 0.28261535027799856], [...","[[0.0, 0.0, 3.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...","[[0.0, 0.0, 3.0], [0.0, 0.0, 3.0], [0.0, 0.0, ...",...,"[[101.6, 91.8, 0.0], [101.6, 84.3, 0.0

In [57]:
file_to_write = f"../004_data/GCN_input/df_balanced_{len(df_balanced)}_normalized_standard.pkl"
df_balanced.to_pickle(file_to_write)

In [15]:
abs(-2+1)

1